# Lab | Tools prompting

**Replace the existing two tools decorators, by creating 3 new ones and adjust the prompts accordingly**


### How to add ad-hoc tool calling capability to LLMs and Chat Models

:::{.callout-caution}

Some models have been fine-tuned for tool calling and provide a dedicated API for tool calling. Generally, such models are better at tool calling than non-fine-tuned models, and are recommended for use cases that require tool calling. Please see the [how to use a chat model to call tools](/docs/how_to/tool_calling) guide for more information.

In this guide, we'll see how to add **ad-hoc** tool calling support to a chat model. This is an alternative method to invoke tools if you're using a model that does not natively support tool calling.

We'll do this by simply writing a prompt that will get the model to invoke the appropriate tools. Here's a diagram of the logic:

<br>

![chain](https://education-team-2020.s3.eu-west-1.amazonaws.com/ai-eng/tool_chain.svg)


## Setup

We'll need to install the following packages:


In [6]:
%pip install langchain-openai



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


If you'd like to use LangSmith, uncomment the below:


In [1]:
import getpass
import os
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

You can select any of the given models for this how-to guide. Keep in mind that most of these models already [support native tool calling](/docs/integrations/chat/), so using the prompting strategy shown here doesn't make sense for these models, and instead you should follow the [how to use a chat model to call tools](/docs/how_to/tool_calling) guide.

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs openaiParams={`model="gpt-4"`} />
```

To illustrate the idea, we'll use `phi3` via Ollama, which does **NOT** have native support for tool calling. If you'd like to use `Ollama` as well follow [these instructions](/docs/integrations/chat/ollama/).


In [14]:
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())


model_2 = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)

In [5]:
from langchain_community.llms import Ollama
from langchain_ollama import OllamaLLM

model = OllamaLLM(model="phi3")

## Create a tool

First, let's create an `add` and `multiply` tools. For more information on creating custom tools, please see [this guide](/docs/how_to/custom_tools).


In [16]:
from langchain_core.tools import tool
import requests
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
from dotenv import load_dotenv, find_dotenv
import os
_ = load_dotenv(find_dotenv())

HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

# Load the model and processor
hf_model = "Salesforce/blip-image-captioning-large"
device = 'cuda' if torch.cuda.is_available() else 'cpu'
processor = BlipProcessor.from_pretrained(hf_model)
model = BlipForConditionalGeneration.from_pretrained(hf_model).to(device)

2024-10-25 13:56:23.683701: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-25 13:56:23.683795: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-25 13:56:23.684944: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-25 13:56:23.693577: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-25 13:56:24.668634: W tensorflow/compiler/tf2

In [9]:
import requests
from PIL import Image

In [17]:
# Image Captioning Tool
@tool
def image_caption(url: str) -> str:
    """
    Given the URL of an image, download it and return a caption describing the image.
    """
    try:

        # Download the image from the URL
        img = Image.open(requests.get(url, stream=True).raw).convert('RGB')

        # Preprocess the image and generate caption
        inputs = processor(img, return_tensors="pt").to(device)
        out = model.generate(**inputs, max_new_tokens=20)
        caption = processor.decode(out[0], skip_special_tokens=True)

        return caption
    except Exception as e:
        return f"Error generating caption: {str(e)}"


# Image Search Tool
@tool
def search_image(query: str) -> str:
    """
    Search for an image on the web based on a query and return a URL.
    """
    try:
        search_url = f"https://google.com/image-search?q={query}"
        response = requests.get(search_url)
        if response.status_code == 200:
            data = response.json()
            return data.get("image_url", "No results found.")
        else:
            return "Failed to search for image."
    except Exception as e:
        return f"Error during image search: {str(e)}"


# SEO Analysis Tool
@tool
def seo_analysis(url: str) -> dict:
    """
    Analyze the SEO of a given URL and return basic SEO metrics.
    """
    try:
        response = requests.get(url)
        if response.status_code == 200:
            # Basic SEO Analysis - Example
            html_content = response.text
            seo_metrics = {
                "title": html_content.split('<title>')[1].split('</title>')[0] if "<title>" in html_content else "No title tag found",
                "meta_description": html_content.split('name="description" content="')[1].split('"')[0] if 'name="description" content="' in html_content else "No meta description found",
                "h1_tags": html_content.count('<h1>'),
                "number_of_links": html_content.count('<a href=')
            }
            return seo_metrics
        else:
            return {"error": "Failed to retrieve the webpage."}
    except Exception as e:
        return {"error": str(e)}


# Register the tools for the agent to use
tools = [image_caption, search_image, seo_analysis]

# Inspect the tools for debugging
for t in tools:
    print("--")
    print(t.name)
    print(t.description)
    print(t.args)

--
image_caption
Given the URL of an image, download it and return a caption describing the image.
{'url': {'title': 'Url', 'type': 'string'}}
--
search_image
Search for an image on the web based on a query and return a URL.
{'query': {'title': 'Query', 'type': 'string'}}
--
seo_analysis
Analyze the SEO of a given URL and return basic SEO metrics.
{'url': {'title': 'Url', 'type': 'string'}}


## Creating our prompt

We'll want to write a prompt that specifies the tools the model has access to, the arguments to those tools, and the desired output format of the model. In this case we'll instruct it to output a JSON blob of the form `{"name": "...", "arguments": {...}}`.


In [22]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import render_text_description

rendered_tools = render_text_description(tools)
print(rendered_tools)

image_caption(url: str) -> str - Given the URL of an image, download it and return a caption describing the image.
search_image(query: str) -> str - Search for an image on the web based on a query and return a URL.
seo_analysis(url: str) -> dict - Analyze the SEO of a given URL and return basic SEO metrics.


In [23]:
system_prompt = f"""\
You are an assistant that has access to the following set of tools.
Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use.
Return your response as a JSON blob with 'name' and 'arguments' keys.

The `arguments` should be a dictionary, with keys corresponding
to the argument names and the values corresponding to the requested values.
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [24]:
chain = prompt | model_2
message = chain.invoke({"input": "search image of a dog"})

# Let's take a look at the output from the model
# if the model is an LLM (not a chat model), the output will be a string.
if isinstance(message, str):
    print(message)
else:  # Otherwise it's a chat model
    print(message.content)

{
  "name": "search_image",
  "arguments": {
    "query": "dog"
  }
}


## Adding an output parser

We'll use the `JsonOutputParser` for parsing our models output to JSON.


In [25]:
from langchain_core.output_parsers import JsonOutputParser

chain = prompt | model_2 | JsonOutputParser()
chain.invoke({"input": "image of a dog"})

{'name': 'search_image', 'arguments': {'query': 'dog'}}

:::{.callout-important}

🎉 Amazing! 🎉 We now instructed our model on how to **request** that a tool be invoked.

Now, let's create some logic to actually run the tool!
:::


## Invoking the tool 🏃

Now that the model can request that a tool be invoked, we need to write a function that can actually invoke
the tool.

The function will select the appropriate tool by name, and pass to it the arguments chosen by the model.


In [26]:
from typing import Any, Dict, Optional, TypedDict

from langchain_core.runnables import RunnableConfig


class ToolCallRequest(TypedDict):
    """A typed dict that shows the inputs into the invoke_tool function."""

    name: str
    arguments: Dict[str, Any]


def invoke_tool(
    tool_call_request: ToolCallRequest, config: Optional[RunnableConfig] = None
):
    """A function that we can use the perform a tool invocation.

    Args:
        tool_call_request: a dict that contains the keys name and arguments.
            The name must match the name of a tool that exists.
            The arguments are the arguments to that tool.
        config: This is configuration information that LangChain uses that contains
            things like callbacks, metadata, etc.See LCEL documentation about RunnableConfig.

    Returns:
        output from the requested tool
    """
    tool_name_to_tool = {tool.name: tool for tool in tools}
    name = tool_call_request["name"]
    requested_tool = tool_name_to_tool[name]
    return requested_tool.invoke(tool_call_request["arguments"], config=config)

Let's test this out 🧪!


## Let's put it together

Let's put it together into a chain that creates a calculator with add and multiplication capabilities.


In [28]:
chain = prompt | model_2 | JsonOutputParser() | invoke_tool
chain.invoke({"input": "https://i.natgeofe.com/n/5d00b0cc-ab95-4522-ad13-7c65b7589e6b/NationalGeographic_748483.jpg?w=718&h=479"})

"Error generating caption: name 'Image' is not defined"

In [25]:
chain = prompt | model_2 | JsonOutputParser() | invoke_tool
chain.invoke({"input": "https://www.thebarefooter.com/faq"})

{'title': 'FAQ | thebarefooter',
 'meta_description': 'Why wear barefoot shoes? Are minimalist shoes good for me? Can barefoot shoes help with my bunions? All the barefoot and minimalist footwear burning questions are answered at The Barefooter FAQ. Come on in and ask any further questions.',
 'h1_tags': 0,
 'number_of_links': 1}

## Returning tool inputs

It can be helpful to return not only tool outputs but also tool inputs. We can easily do this with LCEL by `RunnablePassthrough.assign`-ing the tool output. This will take whatever the input is to the RunnablePassrthrough components (assumed to be a dictionary) and add a key to it while still passing through everything that's currently in the input:


In [20]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    prompt | model | JsonOutputParser() | RunnablePassthrough.assign(output=invoke_tool)
)
chain.invoke({"input": "what's thirteen times 4.14137281"})

{'name': 'multiply',
 'arguments': {'x': 13, 'y': 4.14137281},
 'output': 53.83784653}

## What's next?

This how-to guide shows the "happy path" when the model correctly outputs all the required tool information.

In reality, if you're using more complex tools, you will start encountering errors from the model, especially for models that have not been fine tuned for tool calling and for less capable models.

You will need to be prepared to add strategies to improve the output from the model; e.g.,

1. Provide few shot examples.
2. Add error handling (e.g., catch the exception and feed it back to the LLM to ask it to correct its previous output).
